# random forest

In [ ]:
import pandas as pd
from clearml import Task

In [ ]:
import clearml
clearml.__version__

In [ ]:
# task_ids = Task.query_tasks(project_name='zra/0407')
task_ids = Task.query_tasks(task_filter={'parent': "b5f9c93309b04ebc98497080d4584b0b"})

# task_ids = Task.get_tasks(project_name='zra/0407')
# task_id = task_ids.pop()
# task = Task.get_task(task_id)

In [ ]:

# task = Task.get_task("d3e5bcb46d2a4a25a5df5850fda375d7")

In [ ]:
# {'dataset': {'dataset_file_name': 'dataset_ws60.parquet',
#   'dataset_id': '5dc94de095014553acbe4f011a579241'},
#  'dataset_params': {'analog_group': '150',
#   'business_unit': 'all',
#   'log_target': 'True',
#   'scaler_kwargs': '',
#   'scaling': 'Standard',
#   'split_date': '01.01.2021',
#   'target_column_name_formatter': 'predict_normalized_{}',
#   'window_size': '120'},
#  'kflod_kwargs_params': {'n_splits': '5', 'shuffle': 'False'},
#  'model_kwargs_params': {'alpha': '1.0',
#   'max_depth': '20',
#   'n_estimators': '50'},
#  'model_type_params': {'model_type': 'RandomForestRegressor',
#   'save_kfold_predicts': 'True',
#   'save_model': 'False',
#   'use_kfold': 'False'}}

In [ ]:
def make_dict(task_ids):
    all_data = []
    for task_id in task_ids:
        task = Task.get_task(task_id)
        if task.status!='completed':
            continue
        parameters = task.get_parameters_as_dict()
        scalars = task.get_reported_scalars()
                
        mae = scalars['month']['mae']
        mae = pd.Series(index=mae['x'], data=mae['y'], name=mae['name'])
        rmse = scalars['month']['rmse']
        rmse = pd.Series(index=rmse['x'], data=rmse['y'], name=rmse['name'])

        all_data.append((parameters, (mae, rmse)))
    return all_data


def parse_all_data(all_data):
    res = []

    for x in all_data:
        d = {}
        d['metrics'] = x[1]
        d['bu'] = x[0]['dataset_params']['business_unit']
        d['analog'] = x[0]['dataset_params']['analog_group']
        d['log_target'] = x[0]['dataset_params']['log_target']
        d['scaling'] = x[0]['dataset_params']['scaling']
        d['target_column_name_formatter'] = x[0]['dataset_params']['target_column_name_formatter']
        d['window_size'] = x[0]['dataset_params']['window_size']
        d['use_kfold'] = x[0]['model_type_params']['use_kfold']
        d['model_type'] = x[0]['model_type_params']['model_type']
        d['model_kwargs_params'] = x[0]['model_kwargs_params']
        d['max_depth'] = d['model_kwargs_params']['max_depth']
        d['n_estimators'] = d['model_kwargs_params']['n_estimators']
        res.append(d)
    return res

# def uniq_combinations(l: list, keys=['bu', 'analog']):
#     return set(map(lambda x: tuple([x[key] for key in keys]), l))

def make_res_df(res):
    res_df = pd.DataFrame(list(map(lambda x: {key: value for key, value in x.items() if key!='metrics'}, res)))
    res_df = res_df.merge(pd.concat(list(map(lambda x: x['metrics'][0], res)), axis=1).T.rename({i: f'mae_{i}' for i in range(12)}, axis=1).reset_index(drop=True), left_index=True, right_index=True)
    res_df = res_df.merge(pd.concat(list(map(lambda x: x['metrics'][1], res)), axis=1).T.rename({i: f'rmse_{i}' for i in range(12)}, axis=1).reset_index(drop=True), left_index=True, right_index=True)
    return res_df

def plot_metrics(res_df, groupby='analog'):
    mae_cols = [f'mae_{i}' for i in range(12)]
    rmse_cols = [f'rmse_{i}' for i in range(12)]
    for key, group in res_df.groupby(groupby):
        group[mae_cols].T.plot(legend=False, title=key, ylabel='mae')
        group[rmse_cols].T.plot(legend=False, title=key, ylabel='rmse')


In [ ]:
# all_data_rf = []

# # for task_id in map(lambda x: x.task_id, task_ids):
# for task_id in task_ids:
#     task = Task.get_task(task_id)
#     # if task.status=='completed' and len(task.get_tags())>0:
#     if task.status=='completed' and task.parent=="b5f9c93309b04ebc98497080d4584b0b":
#         parameters = task.get_parameters_as_dict()
#         scalars = task.get_reported_scalars()
                
#         mae = scalars['month']['mae']
#         mae = pd.Series(index=mae['x'], data=mae['y'], name=mae['name'])
#         rmse = scalars['month']['rmse']
#         rmse = pd.Series(index=rmse['x'], data=rmse['y'], name=rmse['name'])

#         all_data_rf.append((parameters, (mae, rmse)))

all_data_rf = make_dict(task_ids)


In [ ]:
all_data_rf.__len__(), task_ids.__len__()

In [ ]:
res_rf = parse_all_data(all_data_rf)
res_df_rf = make_res_df(res_rf)

In [ ]:
mae_cols = [f'mae_{i}' for i in range(12)]
rmse_cols = [f'rmse_{i}' for i in range(12)]

In [ ]:
# for key, group in res_df.groupby('analog'):
#     # print(group.shape)
#     # break
#     group[mae_cols].plot()
#     group[rmse_cols].plot()

In [ ]:
# for key, group in res_df.groupby('analog'):
#     # print(group.shape)
#     # break
#     group[mae_cols].T.plot(legend=False, title=key, ylabel='mae')
#     group[rmse_cols].T.plot(legend=False, title=key, ylabel='rmse')

plot_metrics(res_df_rf)

# catboost

In [ ]:
task_ids_cb = Task.query_tasks(task_filter={'parent': "9950fc7379eb4cd29d95701c713d3444"})

In [ ]:
all_data_cb = make_dict(task_ids_cb)


In [ ]:
all_data_cb.__len__(), task_ids_cb.__len__()

In [ ]:
res_cb = parse_all_data(all_data_cb)
res_df_cb = make_res_df(res_cb)

In [ ]:
res_df_cb

In [ ]:
res_df = pd.concat([res_df_cb, res_df_rf]).reset_index(drop=True)

In [ ]:
res_df

In [ ]:
plot_metrics(res_df)

In [ ]:
df.loc[idxmin]

In [ ]:
res = []

for key, df in res_df.groupby('analog'):
    for metric in ['mae', 'rmse']:
        for month in range(12):
            t = f"{metric}_{month}"
            # print(key, t, df.loc[df[t].idxmin()][['model_type']].values[0])
            idxmin = df[t].idxmin()
            res.append((key, metric, month, idxmin, df.loc[idxmin]['model_type'], df.loc[idxmin]['window_size']))

res = pd.DataFrame(res, columns=['bu', 'metric', 'month', 'idxmin', 'model_type', 'window_size'])

In [ ]:
res.head()

In [ ]:
res['model_type'].value_counts(normalize=True)

In [ ]:
res.groupby(['bu', 'metric']).agg({'model_type': 'value_counts'})

In [ ]:
res.groupby(['month']).agg({'model_type': 'value_counts'})

In [ ]:
res.groupby(['window_size']).agg({'model_type': ['value_counts']}).reset_index()

In [ ]:
126/(126+32+34)

In [ ]:
res

In [ ]:
res_df

In [ ]:
res_df.groupby('analog')[['mae_0', 'mae_1', 'mae_2', 'mae_3',
       'mae_4', 'mae_5', 'mae_6', 'mae_7', 'mae_8', 'mae_9', 'mae_10',
       'mae_11']] \
        .min()

In [ ]:
res_df.groupby('analog')[['rmse_0', 'rmse_1', 'rmse_2', 'rmse_3', 'rmse_4', 'rmse_5',
       'rmse_6', 'rmse_7', 'rmse_8', 'rmse_9', 'rmse_10', 'rmse_11']] \
        .min()

# Посмотрим на предикты, где не было логарифмирования

7c330a572c3f4f9c90c24f133a531b35 - 500
de0210eef5bc4d498c867b9f834ae4f5 - 100
dc0c9989cc714296a75332804513bd30 - 200

In [ ]:
import pickle

In [ ]:
# task = Task.get_task("7c330a572c3f4f9c90c24f133a531b35")
task = Task.get_task("de0210eef5bc4d498c867b9f834ae4f5")



In [ ]:
path = task.artifacts['model_results'].get_local_copy()
with open(path, 'rb') as f:
    data = pickle.load(f)

In [ ]:
target = task.get_parameters_as_dict()['dataset_params']['target_column_name_formatter']

months = len(data)

In [ ]:
target

In [ ]:
month = 9
true_pred = data[month]['X_test_with_predict'][[f"predict_{month}", target.format(month), 'model_predict']]
true_pred
# true_pred.plot()


In [ ]:
task = Task.get_task("dc0c9989cc714296a75332804513bd30")


In [ ]:
path = task.artifacts['model_results'].get_local_copy()
with open(path, 'rb') as f:
    data = pickle.load(f)

In [ ]:
target = task.get_parameters_as_dict()['dataset_params']['target_column_name_formatter']

months = len(data)

In [ ]:
target

In [ ]:
month = 0
true_pred = data[month]['X_test_with_predict'][[f"predict_{month}", target.format(month), 'model_predict']]
true_pred
# true_pred.plot()


In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(true_pred['predict_normalized_0'], true_pred['model_predict'])

# Для Василия

In [ ]:
from functools import reduce

In [ ]:
task = Task.get_task("9950fc7379eb4cd29d95701c713d3444")

In [ ]:
parameters = task.get_parameters()

In [ ]:
parameters

In [ ]:
task_ids_cb = Task.query_tasks(task_filter={'parent': "9950fc7379eb4cd29d95701c713d3444"})
analog_dict = dict()

for task_id in task_ids_cb:
    task = Task.get_task(task_id)
    parameters = task.get_parameters()
    analog = parameters['dataset_params/analog_group']
    if analog in analog_dict.keys():
        continue
    
    path = task.artifacts['model_results'].get_local_copy()
    with open(path, 'rb') as f:
        data = pickle.load(f)

    target = task.get_parameters_as_dict()['dataset_params']['target_column_name_formatter']

    months = len(data)
    preds = []
    for month in range(months):
        df = data[month]['X_test_with_predict'][['cut_date', 'model_predict']].rename({'model_predict': "predict_{}".format(month)}, axis=1)
        preds.append(df)
    preds = reduce(lambda a, b: a.merge(b, on='cut_date'), preds)
    analog_dict[analog] = preds

In [ ]:
# с true

task_ids_cb = Task.query_tasks(task_filter={'parent': "9950fc7379eb4cd29d95701c713d3444"})
analog_dict = dict()

for task_id in task_ids_cb:
    task = Task.get_task(task_id)
    parameters = task.get_parameters()
    analog = parameters['dataset_params/analog_group']
    if analog in analog_dict.keys():
        continue
    
    path = task.artifacts['model_results'].get_local_copy()
    with open(path, 'rb') as f:
        data = pickle.load(f)

    target = task.get_parameters_as_dict()['dataset_params']['target_column_name_formatter']

    months = len(data)
    preds = []
    true = []
    for month in range(months):
        df = data[month]['X_test_with_predict'][['cut_date', 'model_predict']].rename({'model_predict': "predict_{}".format(month)}, axis=1)
        preds.append(df)

        df = data[month]['X_test_with_predict'][['cut_date', f'predict_{month}']].rename({f'predict_{month}': "true_{}".format(month)}, axis=1)
        true.append(df)
    preds = preds + true
    preds = reduce(lambda a, b: a.merge(b, on='cut_date'), preds)
    preds = preds.reset_index().rename(columns={'index': 'DN'})
    preds['DN'] = analog
    analog_dict[analog] = preds

In [ ]:
analog_dict['500']

In [ ]:
# with open(r'D:\gpn\for_vasiliy.pkl', 'wb') as f:
#     pickle.dump(analog_dict, f)



In [ ]:
for key in analog_dict:
    analog_dict[key]['DN'] = key



In [ ]:
pd.concat(analog_dict.values()).reset_index(drop=True).to_excel(r'D:\gpn\for_vasiliy\for_vasiliy2.xlsx', index=0)

# look processing
analysis.ipunb


In [ ]:
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
from typing import List, Dict, Optional, Tuple, Union, Any
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from utils.preprocessing import init_scaler_original, init_scaler, columns_for_deletion

In [ ]:
def split_train_test(
    data_df: pd.DataFrame,
    business_unit: str,
    analog_group: str,
    window_size: int,
    split_date: datetime.date,
    n_predict: int,
    target_column_name_formatter: str,
    log_target: bool = False,
    scaling: Optional[str] = None,
    scaler_kwargs: Optional[Dict[str, Any]] = None,
    drop_not_scalable: bool = True
):
    """_summary_

    Args:
        data_df (pd.DataFrame): Preprocessed dataframe
        analog_group (Union[str, int]): Specific group of analogs.
        window_size (int): number of days used to create the dataset.
        split_date (datetime.date): day relative to which the train and test are divided.
        n_predict (int): What month is the prediction for.
        target_column_name_formatter (str): What value is predicted.
        log_target (bool, optional): whether to logarithm target or not. Defaults to False.
        scaling (Optional[str], optional): whether use specific scaling or not. Defaults to None.
        scaler_kwargs (Optional[Dict[str, Any]], optional): Scaler kwargs if needed. Defaults to None.
        drop_not_scalable (bool, optional): _description_. Defaults to True.

    Raises:
        ValueError: _description_

    Returns:
        _type_: _description_
    """
    if isinstance(split_date, str):
        # split_date = datetime.datetime.strptime(split_date, "%d.%m.%Y").date()
        split_date = pd.to_datetime(split_date)

    valid_scaling = ['MinMaxScaler', 'StandardScaler']
    valid_scaling += ['MinMax', 'Standard']

    target_column_name = target_column_name_formatter.format(n_predict)
    # if target_column_name not in data_df.columns:
    #     raise ValueError(f'Missed columns: {target_column_name}')

    # check scaling
    if scaling and scaling not in valid_scaling:
        raise ValueError("scaling_type must be one of %r." % valid_scaling)

    # tdf = data_df[(data_df['business_unit'] == business_unit) &
    #               (data_df['material_cd'] == analog_group) &
    #               (data_df['window_size'] == window_size)
    #               ]

    print(f"Unique window_sizes: {data_df['window_size'].unique()}")

    tdf = data_df[(data_df['business_unit'] == business_unit)]
    print(tdf.shape[0])
    tdf = tdf[tdf['material_cd'] == analog_group]
    print(tdf.shape[0])
    # tdf = tdf[tdf['window_size'] == window_size]
    # print(tdf.shape[0])


    # split to train and test
    date_to_split = split_date - relativedelta(days=30) * (n_predict + 1)
    date_split_query = tdf['cut_date'].dt.date < date_to_split

    # train_slice = tdf[date_split_query]
    # test_slice = tdf[~date_split_query]

    train_slice = tdf[tdf['cut_date'].dt.date < date_to_split]
    test_slice = tdf[tdf['cut_date'].dt.date >= split_date]

    # work around target
    y_train = train_slice[target_column_name]
    y_test = test_slice[target_column_name]

    if log_target:
        y_train = np.log1p(y_train)
        y_test = np.log1p(y_test)

    if scaler_kwargs is None or len(scaler_kwargs)==0:
        scaler_kwargs = {}
    # scaling, scaler = self.init_scaler(scaling, **scaler_kwargs)
    scaling, scaler = init_scaler_original(scaling, **scaler_kwargs)

    # delete cut_date, mtr_cd, business_unit, w_size, predicts
    columns_to_drop = ['cut_date', 'material_cd',
                       'business_unit', 'window_size']
    columns_to_drop += columns_for_deletion(tdf, startswith='predict')

    # X_train = train_slice.drop(columns_to_drop, axis=1)
    # X_test = test_slice.drop(columns_to_drop, axis=1)
    X_train = train_slice[:]
    X_test = test_slice[:]

    # # delete stock_bmu_count in sum_df dataset
    # if mtr_group == "all":
    #     X_train = X_train.drop(['stock_bmu_count'], axis=1)
    #     X_test = X_test.drop(['stock_bmu_count'], axis=1)
    print(f"Train Shape : {train_slice.shape}, Test Shape {test_slice.shape}")

    if X_train.shape[0]==0:
        return X_train, X_test, y_train, y_test

    # applying scaling
    if scaling:
        columns_list = X_train.columns
        columns_list_ = list(set(X_train.columns) - set(columns_to_drop))

        X_train_ = scaler.fit_transform(X_train[columns_list_])
        X_test_ = scaler.transform(X_test[columns_list_])

        X_train.loc[:, columns_list_] = X_train_
        X_test.loc[:, columns_list_] = X_test_

        # X_train = pd.DataFrame(
        #     X_train, columns=columns_list, index=train_slice.index)
        # X_test = pd.DataFrame(X_test, columns=columns_list,
        #                     index=test_slice.index)

        if not drop_not_scalable:
            for col in columns_to_drop:
                X_train[col] = train_slice[col]
                X_test[col] = test_slice[col]

    return X_train, X_test, y_train, y_test

In [ ]:
month = 0
df = data[month]['X_test_with_predict']
df



In [ ]:
df['predict_0']

In [ ]:
df.shape

# mape

In [ ]:
import pickle

In [ ]:
task_ids = Task.query_tasks(task_filter={'parent': "b5f9c93309b04ebc98497080d4584b0b"})
task_id = task_ids.pop()

In [ ]:
# task = Task.get_task(task_id)
task = Task.get_task("a9d8474580904ad8a70a7db35cb27bdf")

In [ ]:
path = task.artifacts['model_results'].get_local_copy()

In [ ]:
data

In [ ]:
with open(path, 'rb') as f:
    df = pickle.load(f)

In [ ]:
pd.DataFrame(df)

In [ ]:
len(df), len(df[0])

In [ ]:
df[0].keys(), df[1].keys()

In [ ]:
task.get_parameters_as_dict()['dataset_params']['target_column_name_formatter']

In [ ]:
df[1]['X_test_with_predict']#['predict_normalized_']

In [ ]:
def mape(true, pred):
    return 100 * ((true-pred)/true).abs().mean()

In [ ]:
for task_id in task_ids:
    task = Task.get_task(task_id)
    path = task.artifacts['model_results'].get_local_copy()
    with open(path, 'rb') as f:
        data = pickle.load(f)

    target = task.get_parameters_as_dict()['dataset_params']['target_column_name_formatter']

    months = len(data)
    # true_preds = []
    for month in range(months):
        model_true = {target.format(month): "model_true"}
        true_pred = data[month]['X_test_with_predict'].rename(model_true, axis=1)[['model_true', 'model_predict']]
        # true_preds.append(true_pred)
        true = true_pred['model_true']
        pred = true_pred['model_predict']
        mapes.append(mape(true, pred))
    break
        

    
        

In [ ]:
all_data = []

# for task_id in map(lambda x: x.task_id, task_ids):
for task_id in task_ids:
    if task.status!='completed':
        continue
    task = Task.get_task(task_id)
    path = task.artifacts['model_results'].get_local_copy()
    with open(path, 'rb') as f:
        data = pickle.load(f)
    
    parameters = task.get_parameters_as_dict()
    scalars = task.get_reported_scalars()
            
    mae = scalars['month']['mae']
    mae = pd.Series(index=mae['x'], data=mae['y'], name=mae['name'])
    rmse = scalars['month']['rmse']
    rmse = pd.Series(index=rmse['x'], data=rmse['y'], name=rmse['name'])

    target = parameters['dataset_params']['target_column_name_formatter']
    months = len(data)
    mapes = []
    
    for month in range(months):
        model_true = {target.format(month): "model_true"}
        true_pred = data[month]['X_test_with_predict'].rename(model_true, axis=1)[['model_true', 'model_predict']]

        true = true_pred['model_true']
        pred = true_pred['model_predict']
        mapes.append(mape(true, pred))
    mapes = pd.Series(data=mapes, name='mape')

    all_data.append((parameters, (mae, rmse, mapes)))


In [ ]:
def parse_all_data(all_data):
    res = []

    for x in all_data:
        d = {}
        d['metrics'] = x[1]
        d['bu'] = x[0]['dataset_params']['business_unit']
        d['analog'] = x[0]['dataset_params']['analog_group']
        d['log_target'] = x[0]['dataset_params']['log_target']
        d['scaling'] = x[0]['dataset_params']['scaling']
        d['target_column_name_formatter'] = x[0]['dataset_params']['target_column_name_formatter']
        d['window_size'] = x[0]['dataset_params']['window_size']
        d['use_kfold'] = x[0]['model_type_params']['use_kfold']
        d['model_type'] = x[0]['model_type_params']['model_type']
        d['model_kwargs_params'] = x[0]['model_kwargs_params']
        d['max_depth'] = d['model_kwargs_params']['max_depth']
        d['n_estimators'] = d['model_kwargs_params']['n_estimators']
        res.append(d)
    return res

# def uniq_combinations(l: list, keys=['bu', 'analog']):
#     return set(map(lambda x: tuple([x[key] for key in keys]), l))

def make_res_df(res):
    res_df = pd.DataFrame(list(map(lambda x: {key: value for key, value in x.items() if key!='metrics'}, res)))
    res_df = res_df.merge(pd.concat(list(map(lambda x: x['metrics'][0], res)), axis=1).T.rename({i: f'mae_{i}' for i in range(12)}, axis=1).reset_index(drop=True), left_index=True, right_index=True)
    res_df = res_df.merge(pd.concat(list(map(lambda x: x['metrics'][1], res)), axis=1).T.rename({i: f'rmse_{i}' for i in range(12)}, axis=1).reset_index(drop=True), left_index=True, right_index=True)
    return res_df



In [ ]:
res = parse_all_data(all_data)
res_df = make_res_df(res)